<a href="https://colab.research.google.com/github/smi-techie/SQL-Python-Ticket-Predictor/blob/main/Dynamic-Ticket-pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
from datetime import datetime, date, timedelta
import time
import random

DB_FILE = 'event_pricing.db'

def get_db_connection():
    try:
        conn = sqlite3.connect(DB_FILE)

        conn.execute("PRAGMA foreign_keys = ON;")
        print(f"Successfully connected to SQLite database: {DB_FILE}")
        return conn
    except sqlite3.Error as err:
        print(f"Error connecting to SQLite: {err}")
        return None



In [2]:
#Function to Initialize the Database Tables
def initialize_db(conn):
    cursor = conn.cursor()
    try:
        # Create Events table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Events (
            event_id INTEGER PRIMARY KEY AUTOINCREMENT, -- SQLite's auto-incrementing integer primary key
            event_name TEXT NOT NULL,
            event_date TEXT NOT NULL, -- Dates stored as 'YYYY-MM-DD' TEXT in SQLite
            venue_capacity INTEGER NOT NULL,
            base_price REAL NOT NULL, -- Numbers with decimal points stored as REAL in SQLite
            current_price REAL NOT NULL,
            tickets_sold INTEGER DEFAULT 0,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP -- Timestamps stored as TEXT in SQLite
        );
        """)

        # Create Ticket_Sales table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Ticket_Sales (
            sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_id INTEGER NOT NULL,
            purchase_timestamp TEXT DEFAULT CURRENT_TIMESTAMP, -- Timestamps stored as TEXT
            price_paid REAL NOT NULL,
            quantity INTEGER NOT NULL,
            FOREIGN KEY (event_id) REFERENCES Events(event_id)
        );
        """)
        conn.commit() # Save the changes to the database
        print("Database tables created/checked successfully.")
    except sqlite3.Error as err:
        print(f"Error initializing database: {err}")
        conn.rollback() # Undo changes if an error occurs
    finally:
        cursor.close()



In [4]:
# Dynamic Pricing Logic Function
def suggest_price_adjustment(conn, event_id):
    cursor = conn.cursor()
    try:
        sql = """
        SELECT event_name, event_date, venue_capacity, base_price, current_price, tickets_sold
        FROM Events
        WHERE event_id = ?
        """
        cursor.execute(sql, (event_id,))
        event = cursor.fetchone() # Fetch the row

        if not event:
            print(f"Error: Event with ID {event_id} not found for pricing adjustment.")
            return None, "Event not found."


        event_name = event[0]
        event_date_str = event[1]
        venue_capacity = event[2]
        base_price = event[3]
        current_price = event[4]
        tickets_sold = event[5]

        event_date = datetime.strptime(event_date_str, '%Y-%m-%d').date()

        tickets_remaining = venue_capacity - tickets_sold
        sales_percentage = (tickets_sold / venue_capacity) * 100 if venue_capacity > 0 else 0

        # Calculate days until event
        today = date.today()
        days_until_event = (event_date - today).days

        # Pricing Rules
        suggested_price = float(current_price)
        reason = "No change suggested."
        price_change_factor = 0.05 # 5% increase/decrease
        min_price = float(base_price) * 0.5
        max_price = float(base_price) * 2.0

        # Rule 1: Event already passed
        if days_until_event < 0:
            suggested_price = current_price
            reason = "Event has passed."
        # Rule 2: Event is very close (e.g., < 3 days)
        elif days_until_event <= 3:
            if tickets_remaining <= (venue_capacity * 0.10): # Less than 10% tickets left
                suggested_price = current_price * (1 + price_change_factor * 2) # Higher surge pricing
                reason = "Very close to event, high demand (low tickets remaining)."
            elif sales_percentage < 50: # Low sales close to event
                suggested_price = current_price * (1 - price_change_factor * 1.5) # Deeper discount
                reason = "Very close to event, low sales. Trying to sell remaining tickets."
        # Rule 3: Mid-term (e.g., 4-30 days)
        elif days_until_event <= 30:
            if sales_percentage < 30 and tickets_remaining > (venue_capacity * 0.20): # Slow sales, plenty of tickets
                suggested_price = current_price * (1 - price_change_factor) # Discount
                reason = "Sales are slow, offering a discount."
            elif sales_percentage > 70: # Good sales progress
                suggested_price = current_price * (1 + price_change_factor) # Price increase
                reason = "Sales are strong, increasing price."
        # Rule 4: Long-term (e.g., > 30 days)
        else: # Event is far away
            if sales_percentage < 10 and tickets_remaining > (venue_capacity * 0.50):
                suggested_price = current_price * (1 - price_change_factor / 2) # Slight early bird discount
                reason = "Early bird discount to kickstart sales."
            elif sales_percentage > 20: # Early strong interest
                suggested_price = current_price * (1 + price_change_factor / 2) # Early surge pricing
                reason = "Good early interest, slight price increase."


        suggested_price = max(min_price, min(max_price, suggested_price))


        suggested_price = round(suggested_price, 2)


        if abs(suggested_price - current_price) < 0.01:
            reason = "Current price is optimal for current conditions."
            suggested_price = current_price

        return suggested_price, reason

    except sqlite3.Error as err:
        print(f"Error in pricing adjustment for event {event_id}: {err}")
        return None, "Error during calculation."
    finally:
        cursor.close()

def update_event_price(conn, event_id, new_price):
    cursor = conn.cursor()
    try:
        sql = "UPDATE Events SET current_price = ? WHERE event_id = ?"
        cursor.execute(sql, (new_price, event_id))
        conn.commit()
        return True
    except sqlite3.Error as err:
        print(f"Error updating event price: {err}")
        conn.rollback()
        return False
    finally:
        cursor.close()



In [3]:
#Function to Add a New Event
def add_event(conn, name, event_date_str, capacity, base_price):
    cursor = conn.cursor()
    try:

        sql = """
        INSERT INTO Events (event_name, event_date, venue_capacity, base_price, current_price, tickets_sold)
        VALUES (?, ?, ?, ?, ?, ?)
        """

        cursor.execute(sql, (name, event_date_str, capacity, base_price, base_price, 0))
        conn.commit()
        print(f"Event '{name}' added successfully with ID: {cursor.lastrowid}")
        return cursor.lastrowid
    except sqlite3.Error as err:
        print(f"Error adding event: {err}")
        conn.rollback()
        return None
    finally:
        cursor.close()

def simulate_ticket_sale(conn, event_id, current_price):
    cursor = conn.cursor()
    try:
        # Get event details first to check capacity
        cursor.execute("SELECT venue_capacity, tickets_sold FROM Events WHERE event_id = ?", (event_id,))
        event_data = cursor.fetchone()

        if not event_data:

            return False

        venue_capacity, tickets_sold = event_data

        if tickets_sold >= venue_capacity:
            # print(f"Event ID {event_id}: Venue is full, cannot sell more tickets.")
            return False # Cannot sell if full


        quantity_to_sell = random.randint(1, min(5, venue_capacity - tickets_sold))
        if quantity_to_sell == 0:
            return False

        # Record the sale
        sql_insert_sale = """
        INSERT INTO Ticket_Sales (event_id, price_paid, quantity)
        VALUES (?, ?, ?)
        """
        cursor.execute(sql_insert_sale, (event_id, current_price, quantity_to_sell))

        # Update tickets_sold in Events table
        sql_update_event = """
        UPDATE Events
        SET tickets_sold = tickets_sold + ?
        WHERE event_id = ?
        """
        cursor.execute(sql_update_event, (quantity_to_sell, event_id))

        conn.commit()
        # print(f"  Simulated sale: Event ID {event_id}, Qty: {quantity_to_sell}, Price: {current_price:.2f}")
        return True
    except sqlite3.Error as err:
        print(f"Error simulating ticket sale: {err}")
        conn.rollback()
        return False
    finally:
        cursor.close()



In [5]:

if __name__ == "__main__":
    conn = get_db_connection()
    if conn:
        initialize_db(conn)

        print("\n--- Setting up Events ---")

        today_date = date.today()
        event_date1 = (today_date + timedelta(days=90)).strftime('%Y-%m-%d') # ~3 months from now
        event_date2 = (today_date + timedelta(days=30)).strftime('%Y-%m-%d') # ~1 month from now
        event_date3 = (today_date + timedelta(days=5)).strftime('%Y-%m-%d')  # ~5 days from now

        event_id1 = add_event(conn, "Global Music Fest 2025", event_date1, 10000, 75.00)
        event_id2 = add_event(conn, "City Marathon Fun Run", event_date2, 2000, 30.00)
        event_id3 = add_event(conn, "Local Comedy Night", event_date3, 300, 20.00)

        active_event_ids = [e for e in [event_id1, event_id2, event_id3] if e is not None]

        print("\n--- Starting Dynamic Pricing Simulation (Run this cell multiple times or in a loop) ---")


        num_simulation_rounds = 10
        for simulation_round in range(1, num_simulation_rounds + 1):
            print(f"\n--- Simulation Round {simulation_round} ---")

            for event_id in active_event_ids:
                if not event_id:
                    continue

                # Fetch current event details
                cursor = conn.cursor()
                cursor.execute("SELECT event_name, current_price, tickets_sold, venue_capacity, event_date FROM Events WHERE event_id = ?", (event_id,))
                event_details = cursor.fetchone()
                cursor.close()

                if not event_details:
                    print(f"Event {event_id} not found in DB, skipping.")
                    continue

                event_name = event_details[0]
                current_price = event_details[1]
                tickets_sold = event_details[2]
                venue_capacity = event_details[3]
                event_date_str = event_details[4] # Stored as TEXT
                event_date = datetime.strptime(event_date_str, '%Y-%m-%d').date() # Convert to date object

                tickets_remaining = venue_capacity - tickets_sold


                sales_to_simulate = 0
                days_left = (event_date - date.today()).days
                if days_left > 60:
                    sales_to_simulate = random.randint(50, 200) # Far event, moderate sales
                elif days_left > 15:
                     sales_to_simulate = random.randint(10, 50) # Mid-range event, fewer sales
                else: # Close event, fewer sales per round
                     sales_to_simulate = random.randint(1, 10)


                sales_to_simulate = min(sales_to_simulate, tickets_remaining)

                for _ in range(sales_to_simulate):
                    # For each simulated sale, ensuring we use the current price of the event
                    if not simulate_ticket_sale(conn, event_id, current_price):
                        break # Stop simulating sales for this event if it's full


                cursor = conn.cursor()
                cursor.execute("SELECT tickets_sold FROM Events WHERE event_id = ?", (event_id,))
                updated_tickets_sold = cursor.fetchone()[0]
                cursor.close()

                # Get suggested price based on updated data
                new_suggested_price, reason = suggest_price_adjustment(conn, event_id)

                print(f"\n--- {event_name} (ID: {event_id}) ---")
                print(f"  Event Date: {event_date.strftime('%Y-%m-%d')} (Days Left: {days_left if days_left >= 0 else 'Passed'})")
                print(f"  Capacity: {venue_capacity}, Sold: {updated_tickets_sold}, Remaining: {venue_capacity - updated_tickets_sold}")
                print(f"  Current Price: ${current_price:.2f}")

                if new_suggested_price is not None:
                    if new_suggested_price != current_price:
                        print(f"  Suggested New Price: ${new_suggested_price:.2f} ({reason})")
                        update_event_price(conn, event_id, new_suggested_price)
                    else:
                        print(f"  Price remains: ${new_suggested_price:.2f} ({reason})")
                else:
                    print(f"  Could not suggest new price. Reason: {reason}")


    if conn:
        conn.close()
        print("\nSimulation ended. Connection closed.")



Successfully connected to SQLite database: event_pricing.db
Database tables created/checked successfully.

--- Setting up Events ---
Event 'Global Music Fest 2025' added successfully with ID: 1
Event 'City Marathon Fun Run' added successfully with ID: 2
Event 'Local Comedy Night' added successfully with ID: 3

--- Starting Dynamic Pricing Simulation (Run this cell multiple times or in a loop) ---

--- Simulation Round 1 ---

--- Global Music Fest 2025 (ID: 1) ---
  Event Date: 2025-08-19 (Days Left: 90)
  Capacity: 10000, Sold: 181, Remaining: 9819
  Current Price: $75.00
  Suggested New Price: $73.12 (Early bird discount to kickstart sales.)

--- City Marathon Fun Run (ID: 2) ---
  Event Date: 2025-06-20 (Days Left: 30)
  Capacity: 2000, Sold: 76, Remaining: 1924
  Current Price: $30.00
  Suggested New Price: $28.50 (Sales are slow, offering a discount.)

--- Local Comedy Night (ID: 3) ---
  Event Date: 2025-05-26 (Days Left: 5)
  Capacity: 300, Sold: 10, Remaining: 290
  Current Pric